<a href="https://colab.research.google.com/github/jideexploit3211/Davidbots/blob/Joints/Joints.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pybullet imageio

In [ ]:
import pybullet as p
import pybullet_data
import numpy as np
import imageio_ffmpeg
from base64 import b64encode
from IPython.display import HTML

# Connect to PyBullet in DIRECT mode (no GUI)
physicsClient = p.connect(p.DIRECT)
p.setAdditionalSearchPath(pybullet_data.getDataPath())
p.setGravity(0, 0, -10)
planeId = p.loadURDF("plane.urdf")

# Define cube size
cube_size = 0.5

# Create the torso (first cube), positioned higher
torso_collision = p.createCollisionShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3)
torso_visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3, rgbaColor=[1, 0, 0, 1])
torso_id = p.createMultiBody(baseMass=0, baseCollisionShapeIndex=torso_collision, baseVisualShapeIndex=torso_visual, basePosition=[0, 0, cube_size * 1.5])

# Create the arm (second cube), positioned below the torso
arm_collision = p.createCollisionShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3)
arm_visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3, rgbaColor=[0, 1, 0, 1])
arm_id = p.createMultiBody(baseMass=1, baseCollisionShapeIndex=arm_collision, baseVisualShapeIndex=arm_visual, basePosition=[0, -cube_size, cube_size/2])

# Create the leg (third cube), positioned below the torso
leg_collision = p.createCollisionShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3)
leg_visual = p.createVisualShape(p.GEOM_BOX, halfExtents=[cube_size/2]*3, rgbaColor=[0, 0, 1, 1])
leg_id = p.createMultiBody(baseMass=1, baseCollisionShapeIndex=leg_collision, baseVisualShapeIndex=leg_visual, basePosition=[0, cube_size, cube_size/2])

# Create joints
try:
    # Joint between torso and arm
    p.createConstraint(torso_id, -1, arm_id, -1, p.JOINT_REVOLUTE, [0, 0, 1], [0, -cube_size/2, -cube_size/2], [0, cube_size/2, cube_size/2])

    # Joint between torso and leg
    p.createConstraint(torso_id, -1, leg_id, -1, p.JOINT_REVOLUTE, [0, 0, 1], [0, cube_size/2, -cube_size/2], [0, -cube_size/2, cube_size/2])
except Exception as e:
    print(f"Failed to create constraint: {e}")

# Camera parameters
cam_target_pos = [0, 0, cube_size]
cam_distance = 3
cam_yaw, cam_pitch, cam_roll = 50, -30, 0
cam_width, cam_height = 480, 368
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 1], 2, 0.01, 100, 60

# Initialize video
vid = imageio_ffmpeg.write_frames('vid.mp4', (cam_width, cam_height), fps=30)
vid.send(None)  # The first frame of the video must be a null frame.

# Simulate the environment and capture frames
for t in range(240):  # Simulate for 240 steps (8 seconds at 30 fps)
    p.stepSimulation()

    # Capture camera image
    cam_view_matrix = p.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
    cam_projection_matrix = p.computeProjectionMatrixFOV(cam_fov, cam_width / cam_height, cam_near_plane, cam_far_plane)
    image = p.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]

    # Add image to video
    vid.send(np.ascontiguousarray(image))

    # Update yaw for a rotating camera effect
    cam_yaw -= 1

# Close video
vid.close()
p.disconnect()

# Display video in notebook
mp4 = open('vid.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)

Failed to create constraint: createConstraint failed.
